In [17]:
import pandas as pd
import numpy as np
import torch
from trainer import Trainer

from tqdm import trange
from tqdm import tqdm
import torch.nn.functional as F
import os
import os.path as osp
import yaml
from datetime import date

import warnings
warnings.filterwarnings("ignore")

In [19]:
with open("config.yaml", "r") as f:
    args = yaml.safe_load(f)

args["device"] = 'cpu' if args["device"] == 123 else f"cuda:{args['device']}"
args["exp_name"] = f"Date -> {date.today()}. Experiment_{args['sentence_encoder']}_{args['exp_name']}"

In [2]:
# lr = 0.0001 and epochs = 8001

datasets2 = ["ogbn_arxiv_ST", "ogbn_arxiv_roberta"]
model_types = ["mlp", "gcn", "gat", "sage"]

final_results2 = dict.fromkeys(datasets2)

for dataset in tqdm(datasets2):
    dataset_name = "ogbn_arxiv"
    sent_encoder = dataset.split("_")[-1]

    results = dict.fromkeys(model_types)

    # training only on 0th mask
    for model_t in model_types:
        trainer = Trainer(dataset_name=dataset_name, sentence_encoder=sent_encoder, model_type=model_t, device=0)
        acc, loss = trainer.train(0)

        test_acc = acc
        test_loss = loss

        results[model_t] = {"Test Accuracy": test_acc,
                            "Test Loss": test_loss}

    final_results2[dataset] = results

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 10 => Train Accuracy : 0.013514256495969915 | Train Loss : 3.7032053470611572
Validation Accuracy : 0.014664921641665827 | Validation Loss : 3.7022478580474854
Epoch 20 => Train Accuracy : 0.019001330532982922 | Train Loss : 3.6954879760742188
Validation Accuracy : 0.020168462028927144 | Validation Loss : 3.694164991378784
Epoch 30 => Train Accuracy : 0.0372439273814891 | Train Loss : 3.686415433883667
Validation Accuracy : 0.03865901540320145 | Validation Loss : 3.684354782104492
Epoch 40 => Train Accuracy : 0.05885134317854433 | Train Loss : 3.675558567047119
Validation Accuracy : 0.0625524346454579 | Validation Loss : 3.672419786453247
Epoch 50 => Train Accuracy : 0.06947361476121881 | Train Loss : 3.6617660522460938
Validation Accuracy : 0.07231786301553743 | Validation Loss : 3.657808303833008
Epoch 60 => Train Accuracy : 0.08119550037936685 | Train Loss : 3.643515110015869
Validation Accuracy : 0.07913017215342796 | Validation Loss : 3.6384568214416504
Epoch 70 => Train Acc

 50%|█████     | 1/2 [16:14<16:14, 974.22s/it]

Epoch 8000 => Train Accuracy : 0.9054441890896295 | Train Loss : 2.8111608028411865
Validation Accuracy : 0.6201550387596899 | Validation Loss : 3.0768208503723145
Epoch 10 => Train Accuracy : 0.108916770213655 | Train Loss : 3.630801200866699
Validation Accuracy : 0.05298835531393671 | Validation Loss : 3.6544156074523926
Epoch 20 => Train Accuracy : 0.1750035737456153 | Train Loss : 3.548478126525879
Validation Accuracy : 0.09711735293130642 | Validation Loss : 3.5801775455474854
Epoch 30 => Train Accuracy : 0.191893645330489 | Train Loss : 3.436904191970825
Validation Accuracy : 0.11788986207590858 | Validation Loss : 3.475226640701294
Epoch 40 => Train Accuracy : 0.19634708217415686 | Train Loss : 3.2985808849334717
Validation Accuracy : 0.12554112554112554 | Validation Loss : 3.341977119445801
Epoch 50 => Train Accuracy : 0.19999780077192905 | Train Loss : 3.165369987487793
Validation Accuracy : 0.132890365448505 | Validation Loss : 3.2002077102661133
Epoch 60 => Train Accuracy : 

100%|██████████| 2/2 [32:28<00:00, 974.11s/it]

Epoch 8000 => Train Accuracy : 0.8711032427617906 | Train Loss : 2.845567226409912
Validation Accuracy : 0.6028725796167657 | Validation Loss : 3.0917880535125732


In [3]:
final_results2_ = {(i, j) : final_results2[i][j] for i in final_results2.keys() for j in final_results2[i].keys()}
df2 = pd.DataFrame.from_dict(final_results2_, orient="index")
df2

Test Accuracy  Test Loss
ogbn_arxiv_ST      mlp        0.678065   1.019554
                   gcn        0.643911   1.193064
                   gat        0.655350   1.094947
                   sage       0.596506   3.094621
ogbn_arxiv_roberta mlp        0.629920   1.233669
                   gcn        0.606033   1.305170
                   gat        0.619797   1.251771
                   sage       0.570171   3.119816

In [2]:
# lr = 0.001 and epochs = 201

datasets = ["cora_ST", "cora_roberta", "pubmed_ST", "pubmed_roberta"]
model_types = ["mlp", "gcn", "gat", "sage"]

final_results = dict.fromkeys(datasets)

for dataset in tqdm(datasets):
    dataset_name = dataset.split("_")[0]
    sent_encoder = dataset.split("_")[1]

    results = dict.fromkeys(model_types)

    for model_t in model_types:
        test_acc = np.zeros(10)
        test_losses = np.zeros(10)

        for idx in range(10):
            trainer = Trainer(dataset_name=dataset_name, sentence_encoder=sent_encoder, model_type=model_t, device=2)
            acc, loss = trainer.train(idx)

            test_acc[idx] = acc
            test_losses[idx] = loss

        avg_test_acc = test_acc.mean()
        std_test_acc = test_acc.std()

        avg_test_loss = test_losses.mean()
        std_test_loss = test_losses.std()

        results[model_t] = {"Test Accuracy (avg)" : avg_test_acc,
                            "Test Accuracy (std)" : std_test_acc,
                            "Test Loss (avg)" : avg_test_loss,
                            "Test Loss (std)" : std_test_loss}

    final_results[dataset] = results

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch 10 => Train Accuracy : 0.5142857142857142 | Train Loss : 1.9042787551879883
Validation Accuracy : 0.408 | Validation Loss : 1.9136849641799927
Epoch 20 => Train Accuracy : 0.7214285714285714 | Train Loss : 1.7963589429855347
Validation Accuracy : 0.424 | Validation Loss : 1.8681035041809082
Epoch 30 => Train Accuracy : 0.7571428571428571 | Train Loss : 1.5578044652938843
Validation Accuracy : 0.446 | Validation Loss : 1.7515228986740112
Epoch 40 => Train Accuracy : 0.8428571428571429 | Train Loss : 1.1844795942306519
Validation Accuracy : 0.524 | Validation Loss : 1.5310735702514648
Epoch 50 => Train Accuracy : 0.8571428571428571 | Train Loss : 0.8001917004585266
Validation Accuracy : 0.562 | Validation Loss : 1.3101729154586792
Epoch 60 => Train Accuracy : 0.8714285714285714 | Train Loss : 0.5440706014633179
Validation Accuracy : 0.6 | Validation Loss : 1.1522902250289917
Epoch 70 => Train Accuracy : 0.9214285714285714 | Train Loss : 0.34802818298339844
Validation Accuracy : 0.6

 25%|██▌       | 1/4 [01:25<04:15, 85.24s/it]

Epoch 170 => Train Accuracy : 1.0 | Train Loss : 1.117101788520813
Validation Accuracy : 0.818 | Validation Loss : 1.3148947954177856
Epoch 180 => Train Accuracy : 1.0 | Train Loss : 1.1162011623382568
Validation Accuracy : 0.81 | Validation Loss : 1.313452124595642
Epoch 190 => Train Accuracy : 1.0 | Train Loss : 1.1153416633605957
Validation Accuracy : 0.812 | Validation Loss : 1.3068785667419434
Epoch 200 => Train Accuracy : 1.0 | Train Loss : 1.1149128675460815
Validation Accuracy : 0.81 | Validation Loss : 1.3061647415161133
Epoch 10 => Train Accuracy : 0.4857142857142857 | Train Loss : 1.678648591041565
Validation Accuracy : 0.286 | Validation Loss : 1.843103289604187
Epoch 20 => Train Accuracy : 0.6714285714285714 | Train Loss : 1.1081514358520508
Validation Accuracy : 0.408 | Validation Loss : 1.6242482662200928
Epoch 30 => Train Accuracy : 0.8357142857142857 | Train Loss : 0.5971606373786926
Validation Accuracy : 0.436 | Validation Loss : 1.578697919845581
Epoch 40 => Train Ac

 50%|█████     | 2/4 [03:43<03:53, 116.53s/it]

Epoch 200 => Train Accuracy : 1.0 | Train Loss : 1.1143980026245117
Validation Accuracy : 0.692 | Validation Loss : 1.3979182243347168
Epoch 10 => Train Accuracy : 0.85 | Train Loss : 1.0462684631347656
Validation Accuracy : 0.556 | Validation Loss : 1.0756628513336182
Epoch 20 => Train Accuracy : 0.9166666666666666 | Train Loss : 0.8957386016845703
Validation Accuracy : 0.612 | Validation Loss : 0.9982067346572876
Epoch 30 => Train Accuracy : 0.9 | Train Loss : 0.5935218334197998
Validation Accuracy : 0.614 | Validation Loss : 0.8657518029212952
Epoch 40 => Train Accuracy : 0.9666666666666667 | Train Loss : 0.3268578350543976
Validation Accuracy : 0.662 | Validation Loss : 0.7318404316902161
Epoch 50 => Train Accuracy : 0.9833333333333333 | Train Loss : 0.1376207023859024
Validation Accuracy : 0.694 | Validation Loss : 0.6966629028320312
Epoch 60 => Train Accuracy : 1.0 | Train Loss : 0.06935428082942963
Validation Accuracy : 0.712 | Validation Loss : 0.7603083848953247
Epoch 70 => Tr

 75%|███████▌  | 3/4 [07:22<02:43, 163.39s/it]

Epoch 200 => Train Accuracy : 1.0 | Train Loss : 0.46297556161880493
Validation Accuracy : 0.726 | Validation Loss : 0.7482830882072449
Epoch 10 => Train Accuracy : 0.8166666666666667 | Train Loss : 0.7420923113822937
Validation Accuracy : 0.538 | Validation Loss : 0.9451391696929932
Epoch 20 => Train Accuracy : 0.95 | Train Loss : 0.31205806136131287
Validation Accuracy : 0.588 | Validation Loss : 0.832476794719696
Epoch 30 => Train Accuracy : 0.9666666666666667 | Train Loss : 0.12643106281757355
Validation Accuracy : 0.642 | Validation Loss : 0.9503268599510193
Epoch 40 => Train Accuracy : 0.9833333333333333 | Train Loss : 0.030195949599146843
Validation Accuracy : 0.596 | Validation Loss : 1.2458207607269287
Epoch 50 => Train Accuracy : 1.0 | Train Loss : 0.01363447867333889
Validation Accuracy : 0.646 | Validation Loss : 1.2929250001907349
Epoch 60 => Train Accuracy : 1.0 | Train Loss : 0.006004286929965019
Validation Accuracy : 0.624 | Validation Loss : 1.5189976692199707
Epoch 70

100%|██████████| 4/4 [11:32<00:00, 173.04s/it]

Epoch 200 => Train Accuracy : 1.0 | Train Loss : 0.46383237838745117
Validation Accuracy : 0.668 | Validation Loss : 0.8394254446029663


In [3]:
final_results_ = {(i, j) : final_results[i][j] for i in final_results.keys() for j in final_results[i].keys()}
df = pd.DataFrame.from_dict(final_results_, orient="index")
df

Test Accuracy (avg)  Test Accuracy (std)  \
cora_ST        mlp              0.634236             0.025340   
               gcn              0.808027             0.008560   
               gat              0.802079             0.013916   
               sage             0.788781             0.012307   
cora_roberta   mlp              0.483172             0.023032   
               gcn              0.741199             0.015559   
               gat              0.715474             0.016438   
               sage             0.730996             0.013119   
pubmed_ST      mlp              0.717033             0.015539   
               gcn              0.783536             0.014780   
               gat              0.764097             0.021697   
               sage             0.738445             0.017235   
pubmed_roberta mlp              0.626711             0.019911   
               gcn              0.752524             0.021767   
               gat              0.734755             0.019984   
               sage             0.695855             0.026996   

                     Test Loss (avg)  Test Loss (std)  
cora_ST        mlp          1.408994         0.133003  
               gcn          0.612563         0.026717  
               gat          0.616978         0.045621  
               sage         1.326239         0.012104  
cora_roberta   mlp          2.718579         0.224491  
               gcn          0.910239         0.068076  
               gat          1.080747         0.078716  
               sage         1.384649         0.012461  
pubmed_ST      mlp          1.024477         0.095613  
               gcn          0.559440         0.034036  
               gat          0.642786         0.057724  
               sage         0.750760         0.018425  
pubmed_roberta mlp          2.215903         0.192584  
               gcn          0.903236         0.101219  
               gat          1.034255         0.128201  
               sage         0.798594         0.029858